In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pysam
import os
import ast
import numpy as np
import json
import collections
from tqdm import tqdm
from Bio.Seq import Seq
import more_itertools as mit

In [2]:
goodElements=['SINE/Alu','LINE/L1','Retroposon/SVA','snRNA']

In [3]:
t2t1 = pd.read_csv('/home/mark/Desktop/MEI_Group/HGSVC3/MYcallSet/T2T_MEI_INS_06-10-2024.csv').set_index("ID")
t2t_mm = t2t1[(t2t1['TE_Designation'].isin(goodElements)) & (t2t1['Tail_Type']!='No_Tail_Type')].copy()
t2t_mm.drop(index=['chr18-64902896-INS-229'], inplace=True)

hg381 = pd.read_csv('/home/mark/Desktop/MEI_Group/HGSVC3/MYcallSet/hg38_MEI_INS_06-10-2024.csv').set_index("ID")
hg38_mm = hg381[(hg381['TE_Designation'].isin(goodElements)) & (hg381['Tail_Type']!='No_Tail_Type')].copy()
hg38_mm.drop(index=['chr18-64697081-INS-229'], inplace=True)

In [4]:
def find_sequence(sequence, subsequence):
    positions = []
    start = 0
    while True:
        start = sequence.find(subsequence, start)
        if start == -1:
            break
        positions.append(start)
        start += 1
    return positions

In [5]:
def repeatmaskerPatternFilter(df):
    df2 = df.copy()
    annotationList=[]
    df2['Unique_Element_Count']='One_Element'
    for row in df2.index:
        if df2.at[row,'TE_Proportion'] == 'NONE':
            continue
        else:

            prodict = ast.literal_eval(str(df2.at[row,'TE_Proportion']))

            if str(df2.at[row,'TE_Designation']) == 'SINE/Alu':
                aluList = [x for x in prodict.keys() if 'ALU' in str(x).upper()]
                if len(aluList)==1:
                    continue
                else:
                    df2.at[row,'Unique_Element_Count']="More_Than_One_Element"
            else:
                continue 
    
    
    for row in df2.index:
        if df2.at[row,'TE_Proportion'] == 'NONE':
            continue
        else:
        
            if df2.at[row,'Unique_Element_Count'] == 'One_Element':

                tempDict = ast.literal_eval(str(df2.at[row,'TE_Proportion']))
                dictLength = len([x for x in tempDict.keys()])

                if str(df2.at[row,'TE_Designation']) == 'SINE/Alu':
                    if len(ast.literal_eval(str(df2.at[row,'TE_Hits'])))>dictLength:
                        df2.at[row,'Unique_Element_Count'] = 'One_Element_ODD'
                    else:
                        continue



                elif str(df2.at[row,'TE_Designation']) == 'LINE/L1':
                    if len(ast.literal_eval(str(df2.at[row,'TE_Hits'])))>dictLength:
                        df2.at[row,'Unique_Element_Count'] = 'One_Element_ODD'
                    else:
                        continue

                else:
                    if len(ast.literal_eval(str(df2.at[row,'TE_Hits'])))>dictLength:
                        df2.at[row,'Unique_Element_Count'] = 'One_Element_ODD'
                    else:
                        continue


            else:
                continue
    return(df2)

In [6]:
def tailCounter(df):
    
    df2= df.copy()
    df2['FILTER_RESULTS']='Good_Row'
    df2['Tail_Begins']='No_Tail_Detected'
    df2['Tail_Type']='No_Tail_Type'
    df2['Tail_Length']=0
    df2['Tail_Seed_Hits']=0
    
    for row in df2.index:
        

        sequence = str(df2.at[row,'ALT']).upper()
        
        tTail = 'TTTTT'
        tTailFlag=0
        trunningTotal = 0
        tTailList=[]
        
        if tTail in sequence:
            taillength=0
            findings = [int(x) for x in find_sequence(sequence, tTail)]
            if len(findings)>0:
                tTailFlag=1
                tmin = min(findings)
                
                iterable = findings
                df2.at[row,'Tail_Seed_Hits']=len(findings)
                groupings = [list(group) for group in mit.consecutive_groups(iterable)]
                flag=0
                for group in groupings:
                    if flag==0:

                        taillength+= len(tTail) + (len(group)-1)
                        groupEnd = max(group)+len(tTail)
                        flag=1

                    else:

                        if min(group)-groupEnd<=4:
                            taillength+= (len(tTail) + (len(group)-1) + abs(min(group)-groupEnd))
                            groupEnd = max(group)+len(tTail)
                        else:
                            continue

                tTailLength = taillength
                
            else:
                pass
                
        
        aTail = 'AAAAA'
        aTailFlag=0
        arunningTotal = 0
        aTailList=[]
        if aTail in sequence:
            taillength=0
            reverseSequence = str(Seq(sequence).reverse_complement())
            findings = [int(x) for x in find_sequence(reverseSequence, tTail)]
            df2.at[row,'Tail_Seed_Hits']=len(findings)
            if len(findings)>0:
                aTailFlag=1
                amin = min(findings)
                iterable = findings
                groupings = [list(group) for group in mit.consecutive_groups(iterable)]
                flag=0
                for group in groupings:
                    if flag==0:

                        taillength+= len(aTail) + (len(group)-1)
                        groupEnd = max(group)+len(aTail)
                        flag=1

                    else:

                        if min(group)-groupEnd<=4:
                            taillength+= (len(aTail) + (len(group)-1) + abs(min(group)-groupEnd))
                            groupEnd = max(group)+len(aTail)
                        else:
                            continue

                aTailLength = taillength
                
            else:
                pass
        
        if tTailFlag>0 and aTailFlag==0:
            
            df2.at[row,'Tail_Begins']=tmin
            df2.at[row,'Tail_Type']='Possible_T-Tail'
            df2.at[row,'Tail_Length']=tTailLength
            
        elif tTailFlag==0 and aTailFlag>0:
            
            df2.at[row,'Tail_Begins']=amin
            df2.at[row,'Tail_Type']='Possible_A-Tail'
            df2.at[row,'Tail_Length']=aTailLength
        
        
        elif aTailFlag>0 and tTailFlag>0:
            
            orientation = str(df2.at[row,'Orientation'])
            
            if tTailLength<aTailLength and amin<tmin:
                df2.at[row,'Tail_Begins']=amin
                df2.at[row,'Tail_Type']='Possible_A-Tail*_and_Possible_T-Tail'
                df2.at[row,'Tail_Length']=aTailLength
                
            elif tTailLength>aTailLength and tmin<amin:
                df2.at[row,'Tail_Begins']=tmin
                df2.at[row,'Tail_Type']='Possible_A-Tail_and_Possible_T-Tail*'
                df2.at[row,'Tail_Length']=tTailLength
                
            else:
                if orientation !='None':
                    
                    if tmin>=50 and amin<50:
                        df2.at[row,'Tail_Begins']=amin
                        df2.at[row,'Tail_Type']='Possible_A-Tail*_and_Possible_T-Tail'
                        df2.at[row,'Tail_Length']=aTailLength
                    elif tmin<50 and amin>=50:
                        df2.at[row,'Tail_Begins']=tmin
                        df2.at[row,'Tail_Type']='Possible_A-Tail_and_Possible_T-Tail*'
                        df2.at[row,'Tail_Length']=tTailLength
                    
                        
                    elif aTailLength<tTailLength and orientation == '-':
                        df2.at[row,'Tail_Begins']=tmin
                        df2.at[row,'Tail_Type']='Possible_A-Tail_and_Possible_T-Tail*'
                        df2.at[row,'Tail_Length']=tTailLength

                    elif aTailLength>tTailLength and orientation == '+':
                        df2.at[row,'Tail_Begins']=amin
                        df2.at[row,'Tail_Type']='Possible_A-Tail*_and_Possible_T-Tail'
                        df2.at[row,'Tail_Length']=aTailLength
                        
                    elif orientation == '+':
                        df2.at[row,'Tail_Begins']=amin
                        df2.at[row,'Tail_Type']='Possible_A-Tail*_and_Possible_T-Tail'
                        df2.at[row,'Tail_Length']=aTailLength
                    
                    
                    elif orientation == '-':
                        df2.at[row,'Tail_Begins']=tmin
                        df2.at[row,'Tail_Type']='Possible_A-Tail_and_Possible_T-Tail*'
                        df2.at[row,'Tail_Length']=tTailLength
                    
                    else:
                        continue
                else:
                    continue
                    
                
        
        else:
            continue
            
    return(df2)
            

In [7]:
def aluLinker(df):
    df2 = df.copy()
    
    for row in df2.index:
        
        threePrimeFlags=0
        
        if df2.at[row,'TE_Designation'] == 'SINE/Alu' and str(df2.at[row,'Tail_Type'])!='No_Tail_Type':
            
            tailDesignation = str(df2.at[row,'Tail_Type'])
            tailStartSite = int(df2.at[row,'Tail_Begins'])
            elements = ast.literal_eval(str(df2.at[row,'TE_Hits']))
            
            if len(elements)==1:
                for element in elements:

                    splitElement = element.split()

                    if abs(int(splitElement[12])-tailStartSite)>=120 and abs(int(splitElement[12])-tailStartSite)<=150:
                        threePrimeFlags+=1
                    else:
                        continue
            else:
                pass
                        
            if threePrimeFlags>0:
                df2.at[row,'FILTER_RESULTS']=['Alu_Linker_Region_Warning']
            else:
                continue
            
            
        else:
            continue
    return(df2)

In [8]:
def finalQuickCheck(df):
    
    df2 = df.copy()
    
    for row in df2.index:
            
            if str(df2.at[row,'TE_Designation']) =='LINE/L1':
                if str(df2.at[row,'Element_Annotation']) == 'L1HS' or str(df2.at[row,'Element_Annotation']) == 'L1PA1' or str(df2.at[row,'Element_Annotation']) == 'L1PA2':
                    continue
                else:
                    if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                        df2.at[row,'FILTER_RESULTS']=['OLDER_LINE_SUBFAMILY']
                    else:
                        newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                        newList.append('OLDER_LINE_SUBFAMILY')
                        df2.at[row,'FILTER_RESULTS']=newList
                    
                    
            elif str(df2.at[row,'TE_Designation']) =='SINE/Alu':
                
                aluList = ast.literal_eval(str(df2.at[row,'TE_Hits']))
                if len(aluList)==1:
                    continue
                else:
                    
                    aluHitList=[]
                    for hit in aluList:
                        splitHit = hit.split()
                        if 'Alu' in str(splitHit[9]):
                            aluHitList.append(splitHit[9])
                        else:
                            continue
                            
                    if len(set(aluHitList))>1:
                        if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                            df2.at[row,'FILTER_RESULTS']=['MULTI-ALU_Hits']
                        else:
                            newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                            newList.append('MULTI-ALU_Hits')
                            df2.at[row,'FILTER_RESULTS']=newList

                    else:
                        continue

    return(df2)

In [9]:
def finalQuickCheck2(df):
    
    df2 = df.copy()
    
    for row in df2.index:
            
            if str(df2.at[row,'TE_Designation']) =='LINE/L1':
                if int(df2.at[row,'SV_Length']) >10000:
                    if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                        df2.at[row,'FILTER_RESULTS']=['LINE_>10kLen']
                    else:
                        newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                        newList.append('LINE_>10kLen')
                        df2.at[row,'FILTER_RESULTS']=newList
                else:
                    pass
                    
                if int(df2.at[row,'Element_Divergence']) >15.0:
                    if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                        df2.at[row,'FILTER_RESULTS']=['LINE_DIVERGENCE']
                    else:
                        newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                        newList.append('LINE_DIVERGENCE')
                        df2.at[row,'FILTER_RESULTS']=newList
                else:
                    continue
                    
                    
            elif str(df2.at[row,'TE_Designation']) =='SINE/Alu':
                
                if int(df2.at[row,'SV_Length']) >500:
                    if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                        df2.at[row,'FILTER_RESULTS']=['ALU_>500Len']
                    else:
                        newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                        newList.append('ALU_>500Len')
                        df2.at[row,'FILTER_RESULTS']=newList
                else:
                    pass
                    
                if int(df2.at[row,'Element_Divergence']) >6.0:
                    if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                        df2.at[row,'FILTER_RESULTS']=['ALU_DIVERGENCE']
                    else:
                        newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                        newList.append('ALU_DIVERGENCE')
                        df2.at[row,'FILTER_RESULTS']=newList
                else:
                    continue
                        
                        
            elif str(df2.at[row,'TE_Designation']) =='Retroposon/SVA':
                if int(df2.at[row,'SV_Length']) >10000:
                    if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                        df2.at[row,'FILTER_RESULTS']=['SVA_>10kLen']
                    else:
                        newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                        newList.append('SVA_>10kLen')
                        df2.at[row,'FILTER_RESULTS']=newList
                else:
                    pass
                    
                if int(df2.at[row,'Element_Divergence']) >15.0:
                    if str(df2.at[row,'FILTER_RESULTS']) == 'Good_Row':
                        df2.at[row,'FILTER_RESULTS']=['SVA_DIVERGENCE']
                    else:
                        newList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                        newList.append('SVA_DIVERGENCE')
                        df2.at[row,'FILTER_RESULTS']=newList
                else:
                    continue
            
            else:
                continue

    return(df2)

In [10]:
def tailCounterCheck(df):
    df2 = df.copy()
    
    for row in df2.index:
        if df2.at[row,'Tail_Type']!='No_Tail_Type':
            flag=0

            if int(df2.at[row,'Tail_Begins'])/int(df2.at[row,'SV_Length'])>.5 or int(df2.at[row,'Tail_Begins'])>50:
                flag+=1
            else:
                pass

            if flag ==0:
                continue
            else:
                flagg='Bad_Tail_Position'
                if df2.at[row,'FILTER_RESULTS']!='Good_Row':
                    tempFlagList = ast.literal_eval(str(df2.at[row,'FILTER_RESULTS']))
                    tempFlagList.append(flagg)
                    df2.at[row,'FILTER_RESULTS'] = tempFlagList
                else:
                    df2.at[row,'FILTER_RESULTS'] = [flagg]
        else:
            continue
    
    return(df2)

In [11]:
def cleanDataTEPercentage(df):
    
    df2 = df.copy() 
    df2['Element_Annotation']='No_Element_Annotation'
    df2['Element_Divergence']=0.0
    df2['TE_Proportion']='NONE'
    df2['TE_Percentage']=0.0
    
    #For each locus
    for row in tqdm(df2.index):
        
        if df2.at[row,'TE_Hits'] == 'NONE' or int(df2.at[row,'SV_Length'])>50000:
            continue
        else:
            

            columnName = 'ALT'

            tempDict={int(x):0 for x in range(1,len(df2.at[row,columnName])+1)}    
            tempDict2={}
            tempDict3={}
            tempDivDict={}

            teHitList = ast.literal_eval(str(df2.at[row,'TE_Hits']))

            for hit in teHitList:

                splitHit = hit.split()
                focusElement = str(splitHit[9])
                tempDict3[focusElement]=str(splitHit[10])

                if focusElement in tempDict2.keys():
                    tempDivDict[focusElement].append(float(splitHit[1]))
                    pass
                else:
                    tempDivDict[focusElement]=[float(splitHit[1])]
                    tempDict2[focusElement]={x:0 for x in range(1,len(df2.at[row,columnName])+1)}
                    pass

                for coordinate in range(int(splitHit[5]), int(splitHit[6])+1):
                    tempDict[coordinate]+=1
                    tempDict2[focusElement][coordinate]+=1

            tePercentage = len([x for x in tempDict.values() if x >0])/len(tempDict)

            df2.at[row,'TE_Percentage']=float(tePercentage)

            #print(tempDict3)
            #print(tempDivDict)

            if len(tempDict2)==1:
                mykey = [x for x in tempDict2.keys()][0]
                df2.at[row,'TE_Designation']=str(tempDict3[mykey])
                df2.at[row,'TE_Proportion']={mykey:tePercentage}
                df2.at[row,'Element_Annotation']=mykey
                df2.at[row,'Element_Divergence']=np.median(tempDivDict[mykey][0])

            else:

                tempDict4 = {x:len([y for y in tempDict2[x].values() if y>0])/len(tempDict) for x in tempDict2.keys()}
                #print(tempDict4)
                maxKey = max(tempDict4, key=tempDict4.get)
                df2.at[row,'TE_Designation']=tempDict3[str(maxKey)]
                df2.at[row,'TE_Proportion']=tempDict4
                df2.at[row,'Element_Annotation']=maxKey
                df2.at[row,'Element_Divergence']=np.median(tempDivDict[maxKey])
        
    return(df2)

In [12]:
def orientationFinder(df):
    df2 = df.copy()
    df2['Orientation']='NONE'
    for row in df2.index:
        if df2.at[row,'TE_Hits'] == 'NONE':
            continue
        else:
            elementAnnotation = str(df2.at[row,'Element_Annotation'])
            sense=0
            antisense=0
            hitList = ast.literal_eval(str(df2.at[row,'TE_Hits']))
            for hit in hitList:
                if str(hit.split()[9]) == elementAnnotation:
                    if str(hit.split()[8])=='+':
                        sense+=1
                    else:
                        antisense+=1
                else:
                    continue
            
            if sense>0 and antisense ==0:
                df2.at[row,'Orientation']='+'
            elif sense==0 and antisense>0:
                df2.at[row,'Orientation']='-'
            elif sense>antisense:
                df2.at[row,'Orientation']='+'
            elif antisense>sense:
                df2.at[row,'Orientation']='-'
            else:
                continue
    return(df2)
    

In [13]:
def findTwinPriming(df):
    df2 = df.copy()
    df2['Twin_Priming_Flag']='NONE'
    
    for row in df2.index:
        if df2.at[row,'TE_Designation']=='LINE/L1':
            allhits = ast.literal_eval(str(df2.at[row,'TE_Hits']))
            orientations = []
            for hit in allhits:
                splitHit=hit.split()
                if str(splitHit[10]) == 'LINE/L1':
                    orientations.append(splitHit[8])
                else:
                    continue
            if len(set(orientations))>1:
                df2.at[row,'Twin_Priming_Flag']='FLAG'
            else:
                continue
            
            
        else:
            continue
    return(df2)

In [14]:
def simpleRepeatCheck(df):
    df2 = df.copy()
    
    for row in df2.index:
        if df2.at[row,'TE_Designation']=='Simple_repeat':
            tempDict = {x:float(y) for x,y in ast.literal_eval(str(df2.at[row,'TE_Proportion'])).items() if ')n' not in x}
            if len(tempDict)>0:
                maxKey = str(max(tempDict, key=tempDict.get)).upper()
                if 'ALU' in maxKey and len(tempDict) == 1:
                    #print(row)
                    df2.at[row,'TE_Designation']= 'SINE/Alu'
                    df2.at[row,'Element_Annotation']= maxKey

                elif'L1' in maxKey and len(tempDict) == 1:
                    #print(row)
                    df2.at[row,'TE_Designation']= 'LINE/L1'
                    df2.at[row,'Element_Annotation']= maxKey

                elif 'SVA' in maxKey and len(tempDict) == 1:
                    #print(row)
                    df2.at[row,'TE_Designation']= 'Retroposon/SVA'
                    df2.at[row,'Element_Annotation']= maxKey

                else:
                    continue
            else:
                continue
        else:
            continue
    return(df2)

In [15]:
hg38_Filtered= simpleRepeatCheck(findTwinPriming(repeatmaskerPatternFilter(finalQuickCheck2(finalQuickCheck(tailCounterCheck(aluLinker(tailCounter(orientationFinder(cleanDataTEPercentage(hg38_mm))))))))))
collections.Counter([str(x) for x in hg38_Filtered['FILTER_RESULTS']])

100%|███████████████████████████████████| 13178/13178 [00:04<00:00, 2862.07it/s]


Counter({'Good_Row': 12951,
         "['OLDER_LINE_SUBFAMILY']": 168,
         "['Alu_Linker_Region_Warning']": 27,
         "['MULTI-ALU_Hits']": 13,
         "['OLDER_LINE_SUBFAMILY', 'LINE_>10kLen']": 11,
         "['Bad_Tail_Position', 'OLDER_LINE_SUBFAMILY']": 4,
         "['LINE_>10kLen']": 2,
         "['Bad_Tail_Position']": 2})

In [16]:
t2t_Filtered= simpleRepeatCheck(findTwinPriming(repeatmaskerPatternFilter(finalQuickCheck2(finalQuickCheck(tailCounterCheck(aluLinker(tailCounter(orientationFinder(cleanDataTEPercentage(t2t_mm))))))))))
collections.Counter([str(x) for x in t2t_Filtered['FILTER_RESULTS']])

100%|███████████████████████████████████| 13299/13299 [00:04<00:00, 2916.15it/s]


Counter({'Good_Row': 13082,
         "['OLDER_LINE_SUBFAMILY']": 164,
         "['Alu_Linker_Region_Warning']": 24,
         "['MULTI-ALU_Hits']": 11,
         "['OLDER_LINE_SUBFAMILY', 'LINE_>10kLen']": 9,
         "['Bad_Tail_Position', 'OLDER_LINE_SUBFAMILY']": 4,
         "['LINE_>10kLen']": 3,
         "['Bad_Tail_Position']": 2})

In [17]:
## SVAs without real tail
dropList = ['chr8-91388239-INS-116',
'chr11-4280780-INS-2959',
'chr11-4235362-INS-2829',
'chr11-113860443-INS-1521',
'chr16-67712971-INS-7705',
'chr7-1070459-INS-1619',
'chr7-1070459-INS-990',
'chr17-2562918-INS-1336',
'chr22-30067271-INS-7056',
'chr5-42981117-INS-465',
'chr3-88522030-INS-341',
'chr7-88887244-INS-82',
'chr6-39382980-INS-62',
'chr7-124426023-INS-771',
'chr8-7982316-INS-204',
'chrX-54761161-INS-3822',
'chr1-184799913-INS-5043',
'chr12-22429457-INS-6640',
'chr16-7875169-INS-313',
'chr11-101465196-INS-116',
'chr4-155957548-INS-3535',
'chr4-65936764-INS-1163',
'chr4-65936764-INS-275',
'chr6-12431858-INS-1151',
'chr4-74765075-INS-1103',
'chr10-65524280-INS-4197',
'chr8-133779515-INS-159',
'chr10-42445463-INS-9209',
'chr15-47279104-INS-88',
'chr6-154022438-INS-836',
'chr4-68536660-INS-265',
'chr6-110600711-INS-4592',
'chr3-99497282-INS-9239',
'chr4-98039409-INS-187',
'chr5-156906062-INS-117',
'chr1-105943242-INS-128',
'chr1-86636118-INS-4909',
'chr9-8102219-INS-81',
'chr1-197715037-INS-7782',
'chrX-84080392-INS-1372',
'chr14-66703614-INS-286',
'chr2-211217829-INS-1144',
'chr16-58331719-INS-70',
'chrX-87678644-INS-141',
'chr6-82792009-INS-52',
'chr5-147219457-INS-3389',
'chr9-63789428-INS-164',
'chr18-21213420-INS-2160',
'chr7-18904725-INS-134',
'chr8-127521585-INS-1704',
'chr2-105015795-INS-2088',
'chr13-19251652-INS-422',
'chr12-11092199-INS-4182',
'chr9-19671057-INS-1374',
'chr6-32526530-INS-847',
'chr6-16950621-INS-618',
'chr11-121260818-INS-520',
'chr10-36139638-INS-1658',
'chr4-152957-INS-5083',
'chr2-212655670-INS-334',
'chr2-55528173-INS-1115',
'chr12-51772720-INS-272',
'chr20-28752592-INS-1039',
'chr6-30471631-INS-2419',
'chrX-85568238-INS-1653',
'chr6-82897765-INS-287',
'chr4-123908867-INS-261',
'chr12-29846984-INS-212',
'chr9-105087647-INS-4457',
'chr7-136498576-INS-1102',
'chr1-8446290-INS-121',
'chr2-117568027-INS-1174',
'chr9-43063424-INS-9432',
'chr12-129334348-INS-551',
'chr21-17991531-INS-123',
'chr6-74223351-INS-1554',
'chr13-83304851-INS-73',
'chr2-5488557-INS-4001',
'chr13-47740184-INS-1452',
'chr17-30156249-INS-814',
'chr17-14491306-INS-405',
'chr4-9032042-INS-90',
'chrX-85368103-INS-93',
'chr5-45900048-INS-5103',
'chr6-122380088-INS-118',
'chr14-19188568-INS-5121',
'chr6-32529437-INS-608',
'chr11-42797902-INS-6153',
'chr15-22005042-INS-1975',
'chr12-51560369-INS-3044',
'chr11-56613564-INS-5508',
'chrY-454726-INS-525',
'chrX-454726-INS-523',
'chr1-196817007-INS-2130',
'chrUn_KI270538v1-84610-INS-181',
'chr8-91521975-INS-4899',
'chr10-69784615-INS-1621',
'chr14-37301903-INS-355',
'chr12-10579694-INS-2044',
'chr15-56937971-INS-120',
'chr14-88456560-INS-2330',
'chrX-135717079-INS-4357',]

In [18]:
with open('/home/mark/Desktop/MEI_Group/HGSVC3/MYcallSet/badhg38.fasta', 'r') as infile:
    for line in infile:
        if '>' in line:
            dropList.append(line.split(">")[1].strip())
        else:
            continue
infile.close()
len(dropList)

184

In [19]:
goodhg38Rows = []
for row in hg38_Filtered[hg38_Filtered['FILTER_RESULTS']=='Good_Row'].index:
    if row in dropList:
        continue
    else:
        goodhg38Rows.append(row)

In [20]:
hg38_Final = hg38_Filtered[hg38_Filtered['FILTER_RESULTS']=='Good_Row'].loc[goodhg38Rows].copy()
T2T_Final = t2t_Filtered[t2t_Filtered['FILTER_RESULTS']=='Good_Row'].copy()

In [21]:
collections.Counter(hg38_Final['Unique_Element_Count'])

Counter({'One_Element': 12364, 'One_Element_ODD': 571})

In [22]:
collections.Counter(T2T_Final['Unique_Element_Count'])

Counter({'One_Element': 12501, 'One_Element_ODD': 581})

In [51]:
for row in T2T_Final[(T2T_Final['Unique_Element_Count']=='One_Element_ODD') & (T2T_Final['Twin_Priming_Flag']=='FLAG')].index:
    print(row)
    print(T2T_Final.at[row,'TE_Proportion'])
    print(T2T_Final.at[row,'TE_Hits'])
    print(T2T_Final.at[row,'Twin_Priming_Flag'])
    print('\n')

chr1-3543353-INS-674
{'(T)n': 0.04296296296296296, 'L1HS': 0.9185185185185185}
['31 0.0 0.0 0.0 chr1-3543353-INS-674 13 41 (634) + (T)n Simple_repeat 1 29 (0) 45790', '576 0.7 0.0 0.0 chr1-3543353-INS-674 56 508 (167) C L1HS LINE/L1 (1) 6154 5702 45791', '197 1.2 0.0 0.0 chr1-3543353-INS-674 509 675 (0) + L1HS LINE/L1 5553 5719 (436) 45791']
FLAG


chr1-35390237-INS-2279
{'L1HS': 0.9947368421052631}
['1159 1.1 0.0 0.0 chr1-35390237-INS-2279 13 1027 (1253) C L1HS LINE/L1 (0) 6155 5164 45789 *', '1605 1.9 0.0 0.0 chr1-35390237-INS-2279 1008 2280 (0) + L1HS LINE/L1 3896 5168 (978) 45789']
FLAG


chr1-55822696-INS-3169
{'L1HS': 0.9930599369085173}
['1167 1.1 0.0 0.0 chr1-55822696-INS-3169 23 1532 (1638) C L1HS LINE/L1 (1) 6154 4645 47114 *', '2040 2.1 0.0 0.0 chr1-55822696-INS-3169 1528 3170 (0) + L1HS LINE/L1 3011 4653 (1493) 47114']
FLAG


chr1-63118146-INS-2789
{'L1HS': 0.9913978494623656}
['1081 1.1 0.0 0.0 chr1-63118146-INS-2789 25 889 (1901) C L1HS LINE/L1 (0) 6155 5308 47522 *', '24

In [23]:
#hg38_Final.to_csv('/home/mark/Desktop/MEI_Group/HGSVC3/MYcallSet/hg38_MEI_INS_06-17-2024_Filtered.csv')
#T2T_Final.to_csv('/home/mark/Desktop/MEI_Group/HGSVC3/MYcallSet/T2T_MEI_INS_06-17-2024_Filtered.csv')